# Plan-and-Execute

This notebook shows how to create a "plan-and-execute" style agent. This is
heavily inspired by the [Plan-and-Solve](https://arxiv.org/abs/2305.04091) paper
as well as the [Baby-AGI](https://github.com/yoheinakajima/babyagi) project.

The core idea is to first come up with a multi-step plan, and then go through
that plan one item at a time. After accomplishing a particular task, you can
then revisit the plan and modify as appropriate.

This compares to a typical [ReAct](https://arxiv.org/abs/2210.03629) style agent
where you think one step at a time. The advantages of this "plan-and-execute"
style agent are:

1. Explicit long term planning (which even really strong LLMs can struggle with)
2. Ability to use smaller/weaker models for the execution step, only using
   larger/better models for the planning step

## Setup

First, we need to install the packages required.

```bash
npm install @langchain/langgraph @langchain/openai langchain
```

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the
search tool we will use)

In [1]:
// Deno.env.set("OPENAI_API_KEY", "YOUR_API_KEY")
// Deno.env.set("TAVILY_API_KEY", "YOUR_API_KEY")

Optionally, we can set API key for LangSmith tracing, which will give us
best-in-class observability.

In [2]:
// Deno.env.set("LANGCHAIN_TRACING_V2", "true")
// Deno.env.set("LANGCHAIN_API_KEY", "YOUR_API_KEY")
// Deno.env.set("LANGCHAIN_PROJECT", "YOUR_PROJECT_NAME")

In [3]:
import "dotenv/config";

[Module: null prototype] { default: {} }

## Define Tools

We will first define the tools we want to use. For this simple example, we will
use a built-in search tool via Tavily. However, it is really easy to create your
own tools - see documentation
[here](https://js.langchain.com/docs/modules/agents/tools/dynamic) on how to do
that.

In [4]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tools = [new TavilySearchResults({ maxResults: 3 })];

## Define our Execution Agent

Now we will create the execution agent we want to use to execute tasks. Note
that for this example, we will be using the same execution agent for each task,
but this doesn't HAVE to be the case.

In [5]:
import { pull } from "langchain/hub";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { createOpenAIFunctionsAgent } from "langchain/agents";
// Get the prompt to use - you can modify this!
const prompt = await pull<ChatPromptTemplate>(
  "hwchase17/openai-functions-agent",
);
// Choose the LLM that will drive the agent
const llm = new ChatOpenAI({ modelName: "gpt-4-0125-preview" });
// Construct the OpenAI Functions agent
const agentRunnable = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});

In [6]:
import { createAgentExecutor } from "@langchain/langgraph/prebuilt";

const agentExecutor = createAgentExecutor({
  agentRunnable,
  tools,
});

In [7]:
await agentExecutor.invoke({ input: "who is the winner of the us open" });

{
  input: "who is the winner of the us open",
  agentOutcome: {
    returnValues: {
      output: "The winner of the 2023 US Open is Wyndham Clark. He won the 123rd edition of the major, holding off "... 398 more characters
    },
    log: "The winner of the 2023 US Open is Wyndham Clark. He won the 123rd edition of the major, holding off "... 398 more characters
  },
  steps: [
    {
      action: {
        tool: "tavily_search_results_json",
        toolInput: { input: "US Open winner 2023" },
        log: 'Invoking "tavily_search_results_json" with {"input":"US Open winner 2023"}\n',
        messageLog: [ [AIMessage] ]
      },
      observation: `[{"title":"Wyndham Clark wins 2023 US Open, clinching American's first ... - CNN","url":"https://www`... 3219 more characters
    }
  ]
}

## Define the State

Let's now start by defining the state the track for this agent.

First, we will need to track the current plan. Let's represent that as a list of
strings.

Next, we should track previously executed steps. Let's represent that as a list
of tuples (these tuples will contain the step and then the result)

Finally, we need to have some state to represent the final response as well as
the original input.

In [8]:
const planExecuteState = {
  input: {
    value: null,
  },
  plan: {
    value: null,
    default: () => [],
  },
  pastSteps: {
    value: (x, y) => x.concat(y),
    default: () => [],
  },
  response: {
    value: null,
  },
};

## Planning Step

Let's now think about creating the planning step. This will use function calling
to create a plan.

In [9]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";

const plan = zodToJsonSchema(z.object({
  steps: z.array(z.string()).describe(
    "different steps to follow, should be in sorted order",
  ),
}));
const planFunction = {
  name: "plan",
  description: "This tool is used to plan the steps to follow",
  parameters: plan,
};

In [10]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { JsonOutputFunctionsParser } from "langchain/output_parsers";

const plannerPrompt = ChatPromptTemplate.fromTemplate(
  `For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}`,
);
const model = new ChatOpenAI({
  modelName: "gpt-4-0125-preview",
}).bind({
  functions: [planFunction],
  function_call: planFunction,
});
const parserSingle = new JsonOutputFunctionsParser({ argsOnly: true });
const planner = plannerPrompt.pipe(model).pipe(parserSingle);

In [11]:
await planner.invoke({
  objective: "what is the hometown of the current Australia open winner?",
});

{
  steps: [
    "Identify the current Australia Open winner.",
    "Search for the winner's biography or profile.",
    "Locate the hometown information in the profile.",
    "Report the hometown of the current Australia Open winner."
  ]
}

## Re-Plan Step

Now, let's create a step that re-does the plan based on the result of the
previous step.

In [12]:
import { createOpenAIFnRunnable } from "langchain/chains/openai_functions";
import { JsonOutputFunctionsParser } from "langchain/output_parsers";

const response = zodToJsonSchema(z.object({
  response: z.string().describe("Response to user."),
}));
const responseFunction = {
  name: "response",
  description: "Response to user.",
  parameters: response,
};
const replannerPrompt = ChatPromptTemplate.fromTemplate(
  `For the given objective, come up with a simple step by step plan.
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{pastSteps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that and use the 'response' function.
Otherwise, fill out the plan.
Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.`,
);
const parser = new JsonOutputFunctionsParser();
const replanner = createOpenAIFnRunnable({
  functions: [planFunction, responseFunction],
  outputParser: parser,
  llm: new ChatOpenAI({
    modelName: "gpt-4-0125-preview",
  }),
  prompt: replannerPrompt,
});

## Create the Graph

We can now create the graph!

In [13]:
type PlanExecuteState = {
  input: string | null;
  plan: Array<string>;
  pastSteps: Array<string>;
  response: string | null;
};

async function executeStep(
  state: PlanExecuteState,
): Promise<Partial<PlanExecuteState>> {
  const task = state.input;
  const agentResponse = await agentExecutor.invoke({ input: task });
  return { pastSteps: [task, agentResponse.agentOutcome.returnValues.output] };
}

async function planStep(
  state: PlanExecuteState,
): Promise<Partial<PlanExecuteState>> {
  const plan = await planner.invoke({ objective: state.input });
  return { plan: plan.steps };
}

async function replanStep(
  state: PlanExecuteState,
): Promise<Partial<PlanExecuteState>> {
  const output = await replanner.invoke({
    input: state.input,
    plan: state.plan ? state.plan.join("\n") : "",
    pastSteps: state.pastSteps.join("\n"),
  });
  if ("response" in output) {
    return { response: output.response };
  }

  return { plan: output.steps };
}

function shouldEnd(state: PlanExecuteState) {
  if (state.response) {
    return "true";
  }
  return "false";
}

In [14]:
import { END, StateGraph } from "@langchain/langgraph";

const workflow = new StateGraph({
  channels: planExecuteState,
});

// Add the plan node
workflow.addNode("planner", planStep);

// Add the execution step
workflow.addNode("agent", executeStep);

// Add a replan node
workflow.addNode("replan", replanStep);

workflow.setEntryPoint("planner");

// From plan we go to agent
workflow.addEdge("planner", "agent");

// From agent, we replan
workflow.addEdge("agent", "replan");

workflow.addConditionalEdges(
  "replan",
  // Next, we pass in the function that will determine which node is called next.
  shouldEnd,
  {
    "true": END,
    "false": "planner",
  },
);

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile();

In [15]:
const config = { recursionLimit: 50 };
const inputs = {
  input: "what is the hometown of the 2024 Australia open winner?",
};

for await (const event of await app.stream(inputs, config)) {
  console.log(event);
}

{
  planner: {
    plan: [
      "Check for the most recent updates on the 2024 Australia Open winner.",
      "Identify the winner of the 2024 Australia Open.",
      "Research the winner's biography for information on their hometown.",
      "Report the hometown of the 2024 Australia Open winner."
    ]
  }
}
{
  agent: {
    pastSteps: [
      "what is the hometown of the 2024 Australia open winner?",
      "The 2024 Australian Open winner, Jannik Sinner, is from Italy. He became the first Italian man to wi"... 33 more characters
    ]
  }
}
{
  replan: {
    response: "The 2024 Australian Open winner's hometown is not directly provided; however, it is known that Janni"... 156 more characters
  }
}
{
  __end__: {
    input: "what is the hometown of the 2024 Australia open winner?",
    plan: [
      "Check for the most recent updates on the 2024 Australia Open winner.",
      "Identify the winner of the 2024 Australia Open.",
      "Research the winner's biography for information on

> #### See the LangSmith trace [here](https://smith.langchain.com/public/276be79a-3016-4434-83c6-34715b942368/r).